## Setting up the environment: pip install and import

In [1]:
! pip install -q pandas numpy matplotlib shapely geopandas

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from shapely.geometry import Point

## Downloading and opening data

In [3]:
! wget -q https://wifire-data.sdsc.edu/nc/public.php/dav/files/nnSMqWfZAN6Cz6m/new_data/field_data/01_plot_identification.csv
! wget -q https://wifire-data.sdsc.edu/nc/public.php/dav/files/nnSMqWfZAN6Cz6m/new_data/field_data/03_tree.csv

In [4]:
df_plots = pd.read_csv("01_plot_identification.csv")
df_trees = pd.read_csv("03_tree.csv")

## Inspecting the columns of the dataframes

In [5]:
def print_column_names(df):
    cols = 4
    for i, col in enumerate(df.columns):
        last_col = (i + 1) % cols == 0
        end = "\n" if last_col else " "
        print(f"{col:30}", end=end)

In [ ]:
print_column_names(df_plots)

In [ ]:
print_column_names(df_trees)

## Inspecting the rows of the dataframes

In [8]:
#
# Thats a lot of columns
# Lets look at entries for a few of them
#

In [ ]:
#
# Plots dataframe
#
with pd.option_context("display.width", 120):
    cols = ["inventory_id", "plot_coord_x", "plot_coord_y", "site_name", "site_name_label", "inventory_date"]
    print(df_plots[cols])

In [ ]:
#
# Trees dataframe
#
with pd.option_context("display.width", 120):
    # dbh: Diameter at Base Height
    # htlcb: Height to Live Crown Base
    cols = ["inventory_id", "tree_id", "tree_sp_scientific_name", "tree_status", "tree_dbh", "tree_htlcb"]
    print(df_trees[cols])

## Where are the sites?

In [ ]:
print(df_plots[["site_name_label", "plot_coord_x", "plot_coord_y", "plot_coord_srs"]])

In [ ]:
#
# We can use x, y, and srs (Spatial Reference System) to create columns for latitude and longitude
# THIS DOESNT WORK YET
#
gdfs = []
for crs in df_plots["plot_coord_srs"].unique():
    df_crs = df_plots[df_plots["plot_coord_srs"] == crs]
    geometry = [Point(xy) for xy in zip(df_crs["plot_coord_x"], df_crs["plot_coord_y"])]
    gdf = gpd.GeoDataFrame(df_crs, geometry=geometry, crs=f"EPSG:{crs}")
    gdf = gdf.to_crs(epsg=4326)
    gdfs.append(gdf)

# Combine all GeoDataFrames back into one
gdf_combined = pd.concat(gdfs)

# Extract longitude and latitude from geometry
gdf_combined["longitude"] = gdf_combined.geometry.x
gdf_combined["latitude"] = gdf_combined.geometry.y

## How many plots are at each site?

In [ ]:
col = "site_name_label"
for label in df_plots[col].unique():
    nplots = (df_plots[col] == label).sum()
    print(f"{nplots} plots from {label}")

## Adding site name to the tree DataFrame via `merge`

In [ ]:
col = "site_name_label"
if col not in df_trees.columns:
    df_trees = df_trees.merge(df_plots[["inventory_id", col]], on="inventory_id", how="left")

print(df_trees[col])
print()
print("Unique site names:")
print("\n".join(df_trees[col].unique()))

## A few tree-level distributions

In [ ]:
#
# First, lets group the DataFrame by site
#
grouped = df_trees.groupby("site_name_label")
site_names = grouped.groups.keys()
print("\n".join(site_names))

In [ ]:
#
# Lets arrange the grouped columns for a stacked histogram
#
features = ["dbh", "htlcb", "ht"]
tree = {feature:
        grouped[f"tree_{feature}"].apply(list)
        for feature in features}
for feature in tree:
    print("**", feature, "**")
    print(tree[feature])
    print()

In [17]:
#
# Plotting args
#
bins = dict(
    dbh=np.arange(0, 150, step=10),
    htlcb=np.arange(0, 40, step=2),
    ht=np.arange(0, 70, step=5),
)

kwargs = dict(
    stacked=True,
    edgecolor="black",
    linewidth=0.5,
    label=site_names,
)

In [ ]:
#
# Plotting
#
fig, ax = plt.subplots(figsize=(8, 4), ncols=2)
ax[0].hist(tree["dbh"], bins=bins["dbh"], **kwargs)
ax[0].set_xlabel("Diameter at breast height (DBH) [cm]")
ax[0].set_ylabel("Number of trees")
ax[0].legend(fontsize=7)

ax[1].hist(tree["ht"], bins=bins["ht"], **kwargs)
ax[1].set_xlabel("Height (HT) [m]")
ax[1].legend(fontsize=8);

In [ ]:
#
# For 2D, lets compare height and diameter
#
fig, ax = plt.subplots(figsize=(8, 6))
cmap, cmin = plt.cm.winter_r, 0.5
bins = (np.arange(0, 100, step=2), np.arange(0, 60, step=2))
_, _, _, im = ax.hist2d(df_trees["tree_dbh"], df_trees["tree_ht"], bins=bins, cmap=cmap, cmin=cmin)
ax.set_xlabel("Diameter at breast height (DBH) [cm]")
ax.set_ylabel("Height (HT) [m]")
ax.tick_params(top=True, right=True)
cbar = fig.colorbar(im, ax=ax)
cbar.set_label("Number of trees")

---

## Below here is sandboxing

In [ ]:
print(df_trees[ is_site["SDR"] ]["tree_htlcb"])

In [ ]:
! wget -q https://wifire-data.sdsc.edu/nc/public.php/dav/files/nnSMqWfZAN6Cz6m/new_data/field_data/SchemaSummary.csv
df_schema = pd.read_csv("SchemaSummary.csv")
print(df_schema)

In [ ]:
print(df_plots["plot_coord_x"])
print(df_plots["plot_coord_y"])

In [ ]:
import geopandas as gpd
from shapely.geometry import Point

# Example DataFrame in CRS 5070
data = {
    "plot_coord_x": [297590, 297640, 297694, 297646, 297603],
    "plot_coord_y": [4108796, 4108747, 4108903, 4108845, 4108893]
}
df = pd.DataFrame(data)

# Convert DataFrame to GeoDataFrame
geometry = [Point(xy) for xy in zip(df["plot_coord_x"], df["plot_coord_y"])]
gdf = gpd.GeoDataFrame(df, geometry=geometry, crs="EPSG:5070")

# Reproject to WGS84 (latitude and longitude)
gdf = gdf.to_crs(epsg=4326)

# Extract latitude and longitude from geometry
gdf["longitude"] = gdf.geometry.x
gdf["latitude"] = gdf.geometry.y

# Display the updated GeoDataFrame
print(gdf[["latitude", "longitude"]])

In [ ]:
import pandas as pd

# Example DataFrame
data = {
    "plot_coord_x": [-119.4179, -120.0, -121.4944],  # Longitude
    "plot_coord_y": [36.7783, 37.0, 38.5816]        # Latitude
}
df = pd.DataFrame(data)

import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature

# Create a figure and an axis with a specific projection
fig, ax = plt.subplots(figsize=(10, 8), subplot_kw={'projection': ccrs.PlateCarree()})

# Add features to the map
ax.add_feature(cfeature.STATES, edgecolor='black')  # Add state boundaries
ax.add_feature(cfeature.LAND, facecolor='lightgray')  # Add land
ax.add_feature(cfeature.COASTLINE)  # Add coastlines
ax.add_feature(cfeature.BORDERS, linestyle=':')  # Add country borders

# Set the extent to focus on California
ax.set_extent([-125, -114, 32, 42], crs=ccrs.PlateCarree())  # [lon_min, lon_max, lat_min, lat_max]
# ax.set_extent([-225, -14, 12, 72], crs=ccrs.PlateCarree())  # [lon_min, lon_max, lat_min, lat_max]

# Plot the points
ax.scatter(
    gdf['longitude'],
    gdf['latitude'],
    color='red',
    s=50,
    transform=ccrs.PlateCarree(),
    label='Plot Points',
)

# Add title and legend
ax.set_title("Points Overlaid on California Map", fontsize=15)
plt.legend()
plt.show()

In [ ]:
# The internet says Shaver Lake is at:
# Latitude: 37.105831 | Longitude: -119.319528

from pyproj import Transformer

# Example: Assuming the CRS is EPSG:5070 (Conus Albers) and converting to WGS84 (latitude/longitude)
transformer = Transformer.from_crs("EPSG:32611", "EPSG:4326", always_xy=True)
lon, lat = transformer.transform(297590, 4108796)
print(f"Longitude: {lon}, Latitude: {lat}")